In [1]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import svm
from sklearn import tree
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn import tree
from scipy import stats

In [2]:
letter_data = pd.read_csv('letter-recognition.data', header=None)
letter_data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,D,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,C,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,T,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,S,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [4]:
letter_data.columns = ['lettr capital letter', 'x-box horizontal position of box', 'y-box vertical position of box', 'width width of box', 'high height of box', 'onpix total # on pixels', 'x-bar mean x of on pixels in box', 'y-bar mean y of on pixels in box', 'x2bar mean x variance', 'y2bar mean y variance', 'xybar mean x y correlation', 'x2ybr mean of x * x * y', 'xy2br mean of x * y * y', 'x-ege mean edge count left to right', ' xegvy correlation of x-ege with y', 'y-ege mean edge count bottom to top', 'yegvx correlation of y-ege with x']
letter_data = letter_data.dropna()
letter_data[:10]

,lettr capital letter,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,S,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,B,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,A,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,J,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,M,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


In [5]:
def summerize_data(df):
    for column in df.columns:
        print(column)
        if df.dtypes[column] == np.object:
            print(df[column].value_counts())
        else:
            print(df[column].describe())
        print('\n')

In [7]:
summerize_data(letter_data)

lettr capital letter
U    813
D    805
P    803
T    796
M    792
A    789
X    787
Y    786
N    783
Q    783
F    775
G    773
E    768
B    766
V    764
L    761
R    758
I    755
O    753
W    752
S    748
J    747
K    739
C    736
Z    734
H    734
Name: lettr capital letter, dtype: int64


x-box horizontal position of box
count    20000.000000
mean         4.023550
std          1.913212
min          0.000000
25%          3.000000
50%          4.000000
75%          5.000000
max         15.000000
Name: x-box horizontal position of box, dtype: float64


y-box vertical position of box
count    20000.000000
mean         7.035500
std          3.304555
min          0.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         15.000000
Name: y-box vertical position of box, dtype: float64


width width of box
count    20000.000000
mean         5.121850
std          2.014573
min          0.000000
25%          4.000000
50%          5.000000
75%          6.000000
m

In [11]:
letter_data = letter_data.replace({'lettr capital letter': {'O': 1}})
letter_data = letter_data.replace({'lettr capital letter': {'A': 0, 'B':0, 'C':0, 'D':0, 'E':0, 'F':0, 'G':0, 'H':0, 'I':0, 'J':0, 'K':0, 'L':0, 'M':0, 'N':0, 'P':0, 'Q':0, 'R':0, 'S':0, 'T':0, 'U':0, 'V':0, 'W':0, 'X':0, 'Y':0, 'Z':0}})
letter_data[:10]

,lettr capital letter,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,0,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,0,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,0,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
5,0,4,11,5,8,3,8,8,6,9,5,6,6,0,8,9,7
6,0,4,2,5,4,4,8,7,6,6,7,6,6,2,8,7,10
7,0,1,1,3,2,1,8,2,2,2,8,2,8,1,6,2,7
8,0,2,2,4,4,2,10,6,2,6,12,4,8,1,6,1,7
9,0,11,15,13,9,7,13,2,6,2,12,1,9,8,1,1,8


In [12]:
letter_data = letter_data.dropna()
letter_data

,lettr capital letter,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,0,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,0,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,0,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,0,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,0,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,0,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,0,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [13]:
summerize_data(letter_data)

lettr capital letter
count    20000.000000
mean         0.037650
std          0.190353
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: lettr capital letter, dtype: float64


x-box horizontal position of box
count    20000.000000
mean         4.023550
std          1.913212
min          0.000000
25%          3.000000
50%          4.000000
75%          5.000000
max         15.000000
Name: x-box horizontal position of box, dtype: float64


y-box vertical position of box
count    20000.000000
mean         7.035500
std          3.304555
min          0.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         15.000000
Name: y-box vertical position of box, dtype: float64


width width of box
count    20000.000000
mean         5.121850
std          2.014573
min          0.000000
25%          4.000000
50%          5.000000
75%          6.000000
max         15.000000
Name: width width of box, dtype: fl

In [14]:
letter_data2 = letter_data.drop(['lettr capital letter'], axis=1)
letter_data2

,x-box horizontal position of box,y-box vertical position of box,width width of box,high height of box,onpix total # on pixels,x-bar mean x of on pixels in box,y-bar mean y of on pixels in box,x2bar mean x variance,y2bar mean y variance,xybar mean x y correlation,x2ybr mean of x * x * y,xy2br mean of x * y * y,x-ege mean edge count left to right,xegvy correlation of x-ege with y,y-ege mean edge count bottom to top,yegvx correlation of y-ege with x
0,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,2,2,3,3,2,7,7,7,6,6,6,4,2,8,3,7
19996,7,10,8,8,4,4,8,6,9,12,9,13,2,9,3,7
19997,6,9,6,7,5,6,11,3,7,11,9,5,2,12,2,4
19998,2,3,4,2,1,8,7,2,6,10,6,8,1,9,5,8


In [15]:
letter_data['lettr capital letter']

0        0
1        0
2        0
3        0
4        0
        ..
19995    0
19996    0
19997    0
19998    0
19999    0
Name: lettr capital letter, Length: 20000, dtype: int64

In [42]:
#Random Forest
for i in range(5):
    print('trial', i+1)
    letter_rf_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(letter_data2, letter_data['lettr capital letter'], train_size=5000)
        X_train = pd.DataFrame(X_train, columns=X_train.columns)
        X_test = X_test

        for k in range(5):
            letter_rf = RandomForestClassifier(criterion='entropy')
            param_grid = [{'n_estimators': [1024], 'max_features': [1, 2, 4, 6, 8, 12, 16]}]
            #n_list = [1, 2 , 4 , 6, 8, 12, 16, 20]
            letter_rf_grid = GridSearchCV(letter_rf, param_grid, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            letter_rf_grid.fit(X_train, y_train)
            print('mean test accuracy:', letter_rf_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', letter_rf_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', letter_rf_grid.cv_results_['mean_test_f1_micro'])
            letter_rf_results = pd.DataFrame(letter_rf_grid.cv_results_['params'])
            letter_rf_results['score_acc'] = letter_rf_grid.cv_results_['mean_test_accuracy']
            letter_rf_list.append(letter_rf_grid.cv_results_['mean_test_accuracy'])
            letter_rf_cols = letter_rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            letter_rf_results.columns = letter_rf_cols
            letter_rf_results = letter_rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(letter_rf_results)

trial 1
mean test accuracy: [0.9794 0.9846 0.9868 0.9868 0.9872 0.9866 0.9866]
mean test roc auo ovr: [0.99505564 0.99657775 0.99640957 0.99614731 0.99590986 0.99486307
 0.99414305]
mean test f1 micro: [0.9794 0.9846 0.9868 0.9868 0.9872 0.9866 0.9866]
   max_features  n_estimators  score_acc
0             8          1024     0.9872
1             4          1024     0.9868
2             6          1024     0.9868
3            12          1024     0.9866
4            16          1024     0.9866
5             2          1024     0.9846
6             1          1024     0.9794
mean test accuracy: [0.9798 0.9846 0.9864 0.9866 0.9864 0.9862 0.9866]
mean test roc auo ovr: [0.99500621 0.99634776 0.99658669 0.99627889 0.99572462 0.99505842
 0.99409795]
mean test f1 micro: [0.9798 0.9846 0.9864 0.9866 0.9864 0.9862 0.9866]
   max_features  n_estimators  score_acc
0             6          1024     0.9866
1            16          1024     0.9866
2             4          1024     0.9864
3         

mean test accuracy: [0.984  0.9876 0.99   0.9906 0.9902 0.9898 0.9896]
mean test roc auo ovr: [0.99565554 0.99657397 0.9965571  0.99618341 0.99545493 0.993252
 0.98795718]
mean test f1 micro: [0.984  0.9876 0.99   0.9906 0.9902 0.9898 0.9896]
   max_features  n_estimators  score_acc
0             6          1024     0.9906
1             8          1024     0.9902
2             4          1024     0.9900
3            12          1024     0.9898
4            16          1024     0.9896
5             2          1024     0.9876
6             1          1024     0.9840
mean test accuracy: [0.9848 0.9878 0.9898 0.9902 0.9906 0.9898 0.9898]
mean test roc auo ovr: [0.99575937 0.9968965  0.99651689 0.99632785 0.99522564 0.99267834
 0.98669051]
mean test f1 micro: [0.9848 0.9878 0.9898 0.9902 0.9906 0.9898 0.9898]
   max_features  n_estimators  score_acc
0             8          1024     0.9906
1             6          1024     0.9902
2             4          1024     0.9898
3            12     

mean test accuracy: [0.9846 0.9856 0.9866 0.987  0.9868 0.9872 0.9874]
mean test roc auo ovr: [0.9948974  0.9963565  0.99668342 0.99637775 0.99634496 0.99525946
 0.99417203]
mean test f1 micro: [0.9846 0.9856 0.9866 0.987  0.9868 0.9872 0.9874]
   max_features  n_estimators  score_acc
0            16          1024     0.9874
1            12          1024     0.9872
2             6          1024     0.9870
3             8          1024     0.9868
4             4          1024     0.9866
5             2          1024     0.9856
6             1          1024     0.9846
mean test accuracy: [0.9844 0.9858 0.9866 0.987  0.9866 0.9868 0.9876]
mean test roc auo ovr: [0.9953398  0.99644912 0.99655541 0.99649912 0.99599091 0.9952148
 0.99429913]
mean test f1 micro: [0.9844 0.9858 0.9866 0.987  0.9866 0.9868 0.9876]
   max_features  n_estimators  score_acc
0            16          1024     0.9876
1             6          1024     0.9870
2            12          1024     0.9868
3             4    

mean test accuracy: [0.984  0.986  0.988  0.9878 0.9876 0.987  0.9866]
mean test roc auo ovr: [0.99442297 0.99660021 0.99660794 0.99586337 0.99522044 0.99379585
 0.99277501]
mean test f1 micro: [0.984  0.986  0.988  0.9878 0.9876 0.987  0.9866]
   max_features  n_estimators  score_acc
0             4          1024     0.9880
1             6          1024     0.9878
2             8          1024     0.9876
3            12          1024     0.9870
4            16          1024     0.9866
5             2          1024     0.9860
6             1          1024     0.9840
mean test accuracy: [0.9836 0.9868 0.9876 0.9876 0.9878 0.9872 0.987 ]
mean test roc auo ovr: [0.9937114  0.99679806 0.99658986 0.99589563 0.99509006 0.99375683
 0.99300851]
mean test f1 micro: [0.9836 0.9868 0.9876 0.9876 0.9878 0.9872 0.987 ]
   max_features  n_estimators  score_acc
0             8          1024     0.9878
1             4          1024     0.9876
2             6          1024     0.9876
3            12   

mean test accuracy: [0.9848 0.9872 0.9896 0.9896 0.9892 0.989  0.9882]
mean test roc auo ovr: [0.99316346 0.99529369 0.99632082 0.9960067  0.99592531 0.99513606
 0.99443535]
mean test f1 micro: [0.9848 0.9872 0.9896 0.9896 0.9892 0.989  0.9882]
   max_features  n_estimators  score_acc
0             4          1024     0.9896
1             6          1024     0.9896
2             8          1024     0.9892
3            12          1024     0.9890
4            16          1024     0.9882
5             2          1024     0.9872
6             1          1024     0.9848
mean test accuracy: [0.9846 0.987  0.99   0.9894 0.9892 0.9888 0.9884]
mean test roc auo ovr: [0.99330571 0.9960881  0.99646652 0.99603007 0.99562933 0.99503584
 0.99417043]
mean test f1 micro: [0.9846 0.987  0.99   0.9894 0.9892 0.9888 0.9884]
   max_features  n_estimators  score_acc
0             4          1024     0.9900
1             6          1024     0.9894
2             8          1024     0.9892
3            12   

mean test accuracy: [0.9852 0.988  0.9898 0.9898 0.9896 0.9882 0.9878]
mean test roc auo ovr: [0.99503897 0.99624841 0.9964454  0.99576448 0.99537864 0.99469193
 0.99384231]
mean test f1 micro: [0.9852 0.988  0.9898 0.9898 0.9896 0.9882 0.9878]
   max_features  n_estimators  score_acc
0             4          1024     0.9898
1             6          1024     0.9898
2             8          1024     0.9896
3            12          1024     0.9882
4             2          1024     0.9880
5            16          1024     0.9878
6             1          1024     0.9852
mean test accuracy: [0.9852 0.9882 0.9896 0.9898 0.9888 0.9888 0.9882]
mean test roc auo ovr: [0.99504775 0.99628699 0.99631021 0.99576879 0.99517455 0.99488757
 0.99387694]
mean test f1 micro: [0.9852 0.9882 0.9896 0.9898 0.9888 0.9888 0.9882]
   max_features  n_estimators  score_acc
0             6          1024     0.9898
1             4          1024     0.9896
2             8          1024     0.9888
3            12   

mean test accuracy: [0.9824 0.9872 0.9876 0.9876 0.9876 0.9874 0.9882]
mean test roc auo ovr: [0.99451665 0.99584308 0.99617765 0.99567135 0.99537824 0.99450185
 0.99369356]
mean test f1 micro: [0.9824 0.9872 0.9876 0.9876 0.9876 0.9874 0.9882]
   max_features  n_estimators  score_acc
0            16          1024     0.9882
1             8          1024     0.9876
2             4          1024     0.9876
3             6          1024     0.9876
4            12          1024     0.9874
5             2          1024     0.9872
6             1          1024     0.9824
mean test accuracy: [0.983  0.9866 0.9872 0.9874 0.9876 0.987  0.9872]
mean test roc auo ovr: [0.99408734 0.99610363 0.99626943 0.99591414 0.99544634 0.99467358
 0.99388009]
mean test f1 micro: [0.983  0.9866 0.9872 0.9874 0.9876 0.987  0.9872]
   max_features  n_estimators  score_acc
0             8          1024     0.9876
1             6          1024     0.9874
2            16          1024     0.9872
3             4   

mean test accuracy: [0.9848 0.9876 0.9896 0.989  0.9894 0.9896 0.9898]
mean test roc auo ovr: [0.99512983 0.99594007 0.99575758 0.99571579 0.99519043 0.99467161
 0.99431245]
mean test f1 micro: [0.9848 0.9876 0.9896 0.989  0.9894 0.9896 0.9898]
   max_features  n_estimators  score_acc
0            16          1024     0.9898
1             4          1024     0.9896
2            12          1024     0.9896
3             8          1024     0.9894
4             6          1024     0.9890
5             2          1024     0.9876
6             1          1024     0.9848
mean test accuracy: [0.9856 0.9878 0.989  0.989  0.9888 0.9898 0.9888]
mean test roc auo ovr: [0.9948638  0.99591393 0.99597663 0.99565295 0.99529428 0.99519275
 0.99367862]
mean test f1 micro: [0.9856 0.9878 0.989  0.989  0.9888 0.9898 0.9888]
   max_features  n_estimators  score_acc
0            12          1024     0.9898
1             4          1024     0.9890
2             6          1024     0.9890
3             8   

mean test accuracy: [0.9848 0.9886 0.9898 0.99   0.9898 0.9902 0.9902]
mean test roc auo ovr: [0.99514257 0.99640236 0.99649415 0.99622639 0.99554061 0.99438339
 0.99291348]
mean test f1 micro: [0.9848 0.9886 0.9898 0.99   0.9898 0.9902 0.9902]
   max_features  n_estimators  score_acc
0            12          1024     0.9902
1            16          1024     0.9902
2             6          1024     0.9900
3             4          1024     0.9898
4             8          1024     0.9898
5             2          1024     0.9886
6             1          1024     0.9848
mean test accuracy: [0.9848 0.9888 0.9894 0.99   0.9898 0.9902 0.9902]
mean test roc auo ovr: [0.99505582 0.99645835 0.99654541 0.99619704 0.99582718 0.99436328
 0.99308107]
mean test f1 micro: [0.9848 0.9888 0.9894 0.99   0.9898 0.9902 0.9902]
   max_features  n_estimators  score_acc
0            12          1024     0.9902
1            16          1024     0.9902
2             6          1024     0.9900
3             8   

In [43]:
letter_rf_list

[array([0.9838, 0.988 , 0.9894, 0.989 , 0.9886, 0.9882, 0.9876]),
 array([0.9836, 0.9884, 0.9894, 0.989 , 0.989 , 0.988 , 0.9876]),
 array([0.984 , 0.988 , 0.9898, 0.9892, 0.989 , 0.988 , 0.9876]),
 array([0.9838, 0.9876, 0.9898, 0.9896, 0.9888, 0.988 , 0.9876]),
 array([0.9842, 0.9884, 0.9898, 0.9892, 0.9888, 0.9882, 0.9874]),
 array([0.9848, 0.9876, 0.9896, 0.989 , 0.9894, 0.9896, 0.9898]),
 array([0.9856, 0.9878, 0.989 , 0.989 , 0.9888, 0.9898, 0.9888]),
 array([0.9852, 0.9876, 0.9896, 0.9888, 0.9894, 0.9898, 0.9892]),
 array([0.9856, 0.9876, 0.9896, 0.989 , 0.9892, 0.99  , 0.9898]),
 array([0.9852, 0.9874, 0.9892, 0.9888, 0.989 , 0.9894, 0.9894]),
 array([0.983 , 0.9864, 0.9874, 0.9874, 0.9876, 0.9874, 0.9874]),
 array([0.9834, 0.9868, 0.9876, 0.9878, 0.9874, 0.987 , 0.9874]),
 array([0.9828, 0.9864, 0.988 , 0.9876, 0.9876, 0.9878, 0.9876]),
 array([0.9834, 0.987 , 0.9876, 0.9876, 0.9872, 0.9878, 0.9878]),
 array([0.983 , 0.9874, 0.988 , 0.9874, 0.9876, 0.9872, 0.9876]),
 array([0.

In [44]:
letter_rf_grid.cv_results_

{'mean_fit_time': array([1.83014259, 2.03413   , 2.42832351, 2.83630795, 3.2394978 ,
        4.03087292, 4.80684862]),
 'std_fit_time': array([0.00538143, 0.01941981, 0.0179842 , 0.02898809, 0.04394673,
        0.04663784, 0.05714736]),
 'mean_score_time': array([0.3621943 , 0.34219217, 0.3275898 , 0.31998992, 0.32839241,
        0.31019106, 0.30659051]),
 'std_score_time': array([0.00271211, 0.0027176 , 0.00241641, 0.00275632, 0.02133931,
        0.0034872 , 0.00392917]),
 'param_max_features': masked_array(data=[1, 2, 4, 6, 8, 12, 16],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[1024, 1024, 1024, 1024, 1024, 1024, 1024],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_features': 1, 'n_estimators': 1024},
  {'max_features': 2, 'n_estimators': 1024},
  {'max_features': 4, 'n_es

In [45]:
letter_rf_grid.cv_results_['rank_test_accuracy']

array([7, 6, 5, 4, 1, 2, 2])

In [46]:
letter_rf_grid.cv_results_['params'][ np.argmin(letter_rf_grid.cv_results_['rank_test_accuracy'])]

{'max_features': 8, 'n_estimators': 1024}

In [47]:
letter_rf_grid.cv_results_['params'][ np.argmin(letter_rf_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'max_features': 4, 'n_estimators': 1024}

In [48]:
letter_rf_results = pd.DataFrame(letter_rf_grid.cv_results_['params'])
letter_rf_results['score_acc'] = letter_rf_grid.cv_results_['mean_test_accuracy']
letter_rf_cols = letter_rf_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
letter_rf_results.columns = letter_rf_cols
letter_rf_results = letter_rf_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
letter_rf_results

,max_features,n_estimators,score_acc
0,8,1024,0.9902
1,12,1024,0.9900
2,16,1024,0.9900
3,6,1024,0.9898
4,4,1024,0.9896
5,2,1024,0.9892
6,1,1024,0.9854


In [49]:
X_train, X_test, y_train, y_test = train_test_split(letter_data2, letter_data['lettr capital letter'], train_size=5000)
X_train = pd.DataFrame(X_train, columns=X_train.columns)
letter_rf_best = RandomForestClassifier(n_estimators=1024, criterion='entropy', max_features=6)
letter_rf_best.fit(X_train, y_train)
letter_pred_rf = letter_rf_best.predict(X_test)
print('Random Forest:', classification_report(y_test, letter_pred_rf))

Random Forest:               precision    recall  f1-score   support

           0       0.99      1.00      0.99     14424
           1       0.96      0.73      0.83       576

    accuracy                           0.99     15000
   macro avg       0.98      0.87      0.91     15000
weighted avg       0.99      0.99      0.99     15000



In [50]:
#Logistic Regression
for i in range(5):
    print('trial', i+1)
    letter_logreg_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(letter_data2, letter_data['lettr capital letter'], train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)

        for k in range(5):
            pipe = Pipeline([('std', StandardScaler()),
                 ('classifier', linear_model.LogisticRegression())])
            search_space = [{'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['saga'],
                 'classifier__penalty': ['l1', 'l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs'],
                 'classifier__penalty': ['l2'],
                 'classifier__C': np.logspace(-4, 4, 9)},
                {'classifier': [linear_model.LogisticRegression(max_iter=5000)],
                 'classifier__solver': ['lbfgs','saga'],
                 'classifier__penalty': ['none']}]
            letter_logreg_grid = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            letter_logreg_grid.fit(X_train, y_train)
            print('mean test accuracy:', letter_logreg_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', letter_logreg_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', letter_logreg_grid.cv_results_['mean_test_f1_micro'])
            letter_logreg_results = pd.DataFrame(letter_logreg_grid.cv_results_['params'])
            letter_logreg_results['score_acc'] = letter_logreg_grid.cv_results_['mean_test_accuracy']
            letter_logreg_list.append(letter_logreg_grid.cv_results_['mean_test_accuracy'])
            letter_logreg_cols = letter_logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            letter_logreg_results.columns = letter_logreg_cols
            letter_logreg_results = letter_logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(letter_logreg_results)

trial 1
mean test accuracy: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9628
 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.963  0.963
 0.963  0.963  0.9628 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626]
mean test roc auo ovr: [0.5        0.81142264 0.5        0.8320339  0.83882574 0.85772501
 0.84745867 0.8458814  0.83660857 0.83453734 0.83247734 0.8322023
 0.83206758 0.83203952 0.83202829 0.83202829 0.83202829 0.83202829
 0.81141702 0.8320339  0.85773624 0.84587578 0.83453734 0.8322023
 0.8320339  0.83205074 0.83204513 0.83204513 0.83201706]
mean test f1 micro: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9628
 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.963  0.963
 0.963  0.963  0.9628 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9630
1   LogisticRegression(max_iter=5000)     

mean test accuracy: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9628
 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.963  0.963
 0.963  0.963  0.9628 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626]
mean test roc auo ovr: [0.5        0.81141702 0.5        0.8320339  0.83882574 0.85774185
 0.84746429 0.8458814  0.83661418 0.83453734 0.83248295 0.8322023
 0.83207881 0.83204513 0.83202829 0.83202268 0.83202829 0.83202829
 0.81141702 0.8320339  0.85773624 0.84587578 0.83453734 0.8322023
 0.8320339  0.83205074 0.83204513 0.83204513 0.83202268]
mean test f1 micro: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9628
 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626 0.963  0.963
 0.963  0.963  0.9628 0.9626 0.9626 0.9626 0.9626 0.9626 0.9626]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9630
1   LogisticRegression(max_iter=5000)      0.1000 

mean test accuracy: [0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9638 0.9638
 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9644 0.9644
 0.9644 0.9644 0.9638 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636]
mean test roc auo ovr: [0.5        0.84072575 0.5        0.85517126 0.84882501 0.87226
 0.86252215 0.85287966 0.84438351 0.8418064  0.84027699 0.84000406
 0.83978174 0.83978174 0.83977021 0.83977005 0.83977021 0.83977021
 0.84073743 0.8551655  0.87227153 0.85288558 0.84180047 0.83998614
 0.83978719 0.8397699  0.83976429 0.83976429 0.83976445]
mean test f1 micro: [0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9638 0.9638
 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9644 0.9644
 0.9644 0.9644 0.9638 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9644
1   LogisticRegression(max_iter=5000)      0.1000

mean test accuracy: [0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9638 0.9638
 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9644 0.9644
 0.9644 0.9644 0.9638 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636]
mean test roc auo ovr: [0.5        0.84073135 0.5        0.85516566 0.84882501 0.87225424
 0.86252808 0.85287966 0.84437775 0.8418064  0.84028291 0.8400039
 0.83978174 0.83978174 0.83976429 0.83977005 0.83976445 0.83976445
 0.84073743 0.8551655  0.87227153 0.85288558 0.84180047 0.83998614
 0.83978719 0.8397699  0.83976429 0.83976429 0.83977021]
mean test f1 micro: [0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9644 0.9638 0.9638
 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636 0.9644 0.9644
 0.9644 0.9644 0.9638 0.9636 0.9636 0.9636 0.9636 0.9636 0.9636]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9644
1   LogisticRegression(max_iter=5000)      0.10

mean test accuracy: [0.9654 0.9654 0.9654 0.9654 0.9654 0.9654 0.9654 0.9654 0.965  0.965
 0.965  0.965  0.965  0.965  0.965  0.965  0.965  0.965  0.9654 0.9654
 0.9654 0.9654 0.965  0.965  0.965  0.965  0.965  0.965  0.965 ]
mean test roc auo ovr: [0.5        0.80496279 0.5        0.82176681 0.82824138 0.84705257
 0.84024105 0.8347461  0.82698355 0.82520197 0.82350242 0.82325695
 0.82308863 0.82307695 0.82307729 0.82306545 0.82305952 0.82305952
 0.80495687 0.82176072 0.84705849 0.83474576 0.82517829 0.82323259
 0.82305327 0.82307762 0.82307762 0.82307762 0.82305952]
mean test f1 micro: [0.9654 0.9654 0.9654 0.9654 0.9654 0.9654 0.9654 0.9654 0.965  0.965
 0.965  0.965  0.965  0.965  0.965  0.965  0.965  0.965  0.9654 0.9654
 0.9654 0.9654 0.965  0.965  0.965  0.965  0.965  0.965  0.965 ]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9654
1   LogisticRegression(max_iter=5000)      0.100

...

mean test accuracy: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9626 0.9624
 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.963  0.963
 0.963  0.963  0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624]
mean test roc auo ovr: [0.5        0.82599983 0.5        0.83890432 0.84038337 0.85545733
 0.8438663  0.84459038 0.83866858 0.8370071  0.83553647 0.83542982
 0.83527266 0.83524459 0.83519969 0.83519969 0.8352053  0.8352053
 0.82600544 0.83892116 0.85545171 0.84461284 0.83700149 0.8354186
 0.83523336 0.83519969 0.83519969 0.83519969 0.8352053 ]
mean test f1 micro: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9626 0.9624
 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.963  0.963
 0.963  0.963  0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9630
1   LogisticRegression(max_iter=5000)      0.1000 

mean test accuracy: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9626 0.9624
 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.963  0.963
 0.963  0.963  0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624]
mean test roc auo ovr: [0.5        0.82600544 0.5        0.83891555 0.84038337 0.85545733
 0.84386068 0.84458477 0.83867419 0.83700149 0.8355477  0.83542982
 0.83527266 0.8352502  0.83519969 0.8352053  0.8352053  0.8352053
 0.82600544 0.83892116 0.85545171 0.84461284 0.83700149 0.8354186
 0.83523336 0.83519969 0.83519969 0.83519969 0.8352053 ]
mean test f1 micro: [0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.963  0.9626 0.9624
 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.963  0.963
 0.963  0.963  0.9624 0.9624 0.9624 0.9624 0.9624 0.9624 0.9624]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9630
1   LogisticRegression(max_iter=5000)      0.1000 

mean test accuracy: [0.9616 0.9616 0.9616 0.9616 0.9616 0.9616 0.9616 0.9614 0.961  0.9608
 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606 0.9616 0.9616
 0.9616 0.9614 0.9608 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606]
mean test roc auo ovr: [0.5        0.83484186 0.5        0.84586126 0.85225628 0.85541819
 0.84530903 0.84444269 0.84017455 0.83782248 0.83669099 0.83653516
 0.83645525 0.83641189 0.83640655 0.83641189 0.83640655 0.83641202
 0.83484186 0.84585593 0.85540725 0.84444816 0.83783356 0.83653543
 0.83639574 0.83643911 0.83643377 0.83643377 0.83641189]
mean test f1 micro: [0.9616 0.9616 0.9616 0.9616 0.9616 0.9616 0.9616 0.9614 0.961  0.9608
 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606 0.9616 0.9616
 0.9616 0.9614 0.9608 0.9606 0.9606 0.9606 0.9606 0.9606 0.9606]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9616
1   LogisticRegression(max_iter=5000)      0.1

mean test accuracy: [0.9586 0.9586 0.9586 0.9586 0.9586 0.9586 0.9584 0.9578 0.957  0.9568
 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9586 0.9586
 0.9586 0.9578 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568]
mean test roc auo ovr: [0.5        0.82873003 0.5        0.84206979 0.85161828 0.85447683
 0.84730168 0.84668242 0.8434814  0.84266189 0.84202481 0.84197995
 0.84192862 0.84188816 0.84190828 0.84190828 0.84190319 0.84190319
 0.82871027 0.84209476 0.85448192 0.84670265 0.84267183 0.84198469
 0.84191336 0.84186782 0.84187279 0.84187279 0.84189302]
mean test f1 micro: [0.9586 0.9586 0.9586 0.9586 0.9586 0.9586 0.9584 0.9578 0.957  0.9568
 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9586 0.9586
 0.9586 0.9578 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9586
1   LogisticRegression(max_iter=5000)      0.0

mean test accuracy: [0.9586 0.9586 0.9586 0.9586 0.9586 0.9586 0.9584 0.9578 0.957  0.9568
 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9586 0.9586
 0.9586 0.9578 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568]
mean test roc auo ovr: [0.5        0.82873003 0.5        0.84208482 0.85161828 0.85448689
 0.84730654 0.84669248 0.8434814  0.84266178 0.84202481 0.84197995
 0.84191845 0.84190839 0.84190828 0.84190319 0.84190319 0.8418981
 0.82871027 0.84209476 0.85448192 0.84670265 0.84267183 0.84198469
 0.84191336 0.84186782 0.84187279 0.84187279 0.84189822]
mean test f1 micro: [0.9586 0.9586 0.9586 0.9586 0.9586 0.9586 0.9584 0.9578 0.957  0.9568
 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9586 0.9586
 0.9586 0.9578 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568 0.9568]
                           classifier           C penalty solver  score_acc
0   LogisticRegression(max_iter=5000)      0.0001      l1   saga     0.9586
1   LogisticRegression(max_iter=5000)      0.01

In [51]:
letter_logreg_list

[array([0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9608, 0.9608, 0.9608, 0.9608, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604]),
 array([0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9608, 0.9608, 0.9608, 0.9608, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604]),
 array([0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9608, 0.9608, 0.9608, 0.9608, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604]),
 array([0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608, 0.9608,
        0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604, 0.9604,
        0.9604, 0.9604, 0.9608, 0.9608, 0.9608, 0.9608, 0.

In [52]:
letter_logreg_grid.cv_results_

{'mean_fit_time': array([0.00639696, 0.01999903, 0.00719981, 0.02060571, 0.01999531,
        0.02199869, 0.02499905, 0.02499914, 0.03299541, 0.03139892,
        0.0360043 , 0.03240328, 0.03779907, 0.0317955 , 0.0352047 ,
        0.0312016 , 0.0357986 , 0.0329987 , 0.01279955, 0.01059971,
        0.01179972, 0.01639957, 0.01859922, 0.01939912, 0.0195991 ,
        0.01959925, 0.02019906, 0.01919909, 0.03120198]),
 'std_fit_time': array([0.0004919 , 0.00109736, 0.00147559, 0.00150044, 0.00244961,
        0.00167269, 0.00189735, 0.00275697, 0.00438155, 0.00265395,
        0.00362883, 0.00293812, 0.00249022, 0.00213616, 0.00270799,
        0.00371164, 0.00354413, 0.00178878, 0.00074853, 0.00049006,
        0.0003999 , 0.00079986, 0.00080011, 0.00049021, 0.0004899 ,
        0.00049002, 0.00074858, 0.00039973, 0.00248263]),
 'mean_score_time': array([0.00520539, 0.00579696, 0.00519395, 0.00559945, 0.00540056,
        0.00560026, 0.0056004 , 0.00599985, 0.00500002, 0.00579963,
        0.005399

In [53]:
letter_logreg_grid.cv_results_['rank_test_accuracy']

array([ 1,  1,  1,  1,  1,  1, 10, 11, 13, 14, 14, 14, 14, 14, 14, 14, 14,
       14,  1,  1,  1, 11, 14, 14, 14, 14, 14, 14, 14])

In [54]:
letter_logreg_grid.cv_results_['params'][ np.argmin(letter_logreg_grid.cv_results_['rank_test_accuracy'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 0.0001,
 'classifier__penalty': 'l1',
 'classifier__solver': 'saga'}

In [55]:
letter_logreg_grid.cv_results_['params'][ np.argmin(letter_logreg_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'classifier': LogisticRegression(max_iter=5000),
 'classifier__C': 0.01,
 'classifier__penalty': 'l2',
 'classifier__solver': 'saga'}

In [56]:
letter_logreg_results = pd.DataFrame(letter_logreg_grid.cv_results_['params'])
letter_logreg_results['score_acc'] = letter_logreg_grid.cv_results_['mean_test_accuracy']
letter_logreg_cols = letter_logreg_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
letter_logreg_results.columns = letter_logreg_cols
letter_logreg_results = letter_logreg_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
letter_logreg_results

,classifier,C,penalty,solver,score_acc
0,LogisticRegression(max_iter=5000),0.0001,l1,saga,0.9586
1,LogisticRegression(max_iter=5000),0.0100,l2,saga,0.9586
2,LogisticRegression(max_iter=5000),0.0100,l2,lbfgs,0.9586
3,LogisticRegression(max_iter=5000),0.0001,l2,saga,0.9586
4,LogisticRegression(max_iter=5000),0.0010,l2,lbfgs,0.9586
5,LogisticRegression(max_iter=5000),0.0001,l2,lbfgs,0.9586
6,LogisticRegression(max_iter=5000),0.0100,l1,saga,0.9586
7,LogisticRegression(max_iter=5000),0.0010,l2,saga,0.9586
8,LogisticRegression(max_iter=5000),0.0010,l1,saga,0.9586
9,LogisticRegression(max_iter=5000),0.1000,l1,saga,0.9584


In [57]:
X_train, X_test, y_train, y_test = train_test_split(letter_data2, letter_data['lettr capital letter'], train_size=5000)
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = sc.transform(X_test)
letter_logreg_best = linear_model.LogisticRegression(penalty='l2', C=.01, solver='saga', max_iter=5000)
letter_logreg_best.fit(X_train, y_train)
letter_pred_logreg = letter_logreg_best.predict(X_test)
print('Logistic Regression:', classification_report(y_test, letter_pred_logreg))

Logistic Regression:               precision    recall  f1-score   support

           0       0.96      1.00      0.98     14426
           1       0.00      0.00      0.00       574

    accuracy                           0.96     15000
   macro avg       0.48      0.50      0.49     15000
weighted avg       0.92      0.96      0.94     15000



C:\Users\roxmy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\roxmy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\roxmy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [58]:
# KNN
for i in range(5):
    print('trial', i+1)
    letter_knn_list = []

    for j in range(5):
        X_train, X_test, y_train, y_test = train_test_split(letter_data2, letter_data['lettr capital letter'], train_size=5000)
        sc = StandardScaler()
        X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
        X_test = sc.transform(X_test)

        for k in range(5):
            letter_knn = KNeighborsClassifier(weights='distance')
            k_list = [1, 5 , 9 , 13, 17, 21, 25, 29, 33, 37, 41, 45, 49, 53, 57, 61, 65, 69, 73, 77, 81, 85, 89, 93, 97, 101]
            letter_knn_grid = GridSearchCV(letter_knn, {'n_neighbors':k_list}, cv=StratifiedKFold(n_splits=5), scoring=['accuracy', 'roc_auc_ovr', 'f1_micro'], refit=False,
                   verbose=0, return_train_score=True)
            letter_knn_grid.fit(X_train, y_train)
            print('mean test accuracy:', letter_knn_grid.cv_results_['mean_test_accuracy'])
            print('mean test roc auo ovr:', letter_knn_grid.cv_results_['mean_test_roc_auc_ovr'])
            print('mean test f1 micro:', letter_knn_grid.cv_results_['mean_test_f1_micro'])
            letter_knn_results = pd.DataFrame(letter_knn_grid.cv_results_['params'])
            letter_knn_results['score_acc'] = letter_knn_grid.cv_results_['mean_test_accuracy']
            letter_knn_list.append(letter_knn_grid.cv_results_['mean_test_accuracy'])
            letter_knn_cols = letter_knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
            letter_knn_results.columns = letter_knn_cols
            letter_knn_results = letter_knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
            print(letter_knn_results)

trial 1
mean test accuracy: [0.989  0.9884 0.9864 0.9854 0.9852 0.985  0.9848 0.9848 0.9848 0.9836
 0.9842 0.984  0.9832 0.9822 0.9814 0.9804 0.9792 0.9782 0.977  0.9758
 0.9746 0.9746 0.9734 0.9728 0.972  0.9714]
mean test roc auo ovr: [0.95131305 0.98424882 0.98865029 0.98756702 0.98966517 0.99141865
 0.99067458 0.99196849 0.99122989 0.99058978 0.98975818 0.98958858
 0.9885436  0.987619   0.98707736 0.98660138 0.98649743 0.98586826
 0.98533209 0.98479593 0.98421053 0.98335157 0.98254185 0.98184156
 0.98127257 0.98061604]
mean test f1 micro: [0.989  0.9884 0.9864 0.9854 0.9852 0.985  0.9848 0.9848 0.9848 0.9836
 0.9842 0.984  0.9832 0.9822 0.9814 0.9804 0.9792 0.9782 0.977  0.9758
 0.9746 0.9746 0.9734 0.9728 0.972  0.9714]
    n_neighbors  score_acc
0             1     0.9890
1             5     0.9884
2             9     0.9864
3            13     0.9854
4            17     0.9852
5            21     0.9850
6            25     0.9848
7            33     0.9848
8            29     0.

mean test accuracy: [0.9876 0.988  0.9868 0.987  0.9856 0.985  0.984  0.9836 0.9834 0.9832
 0.9824 0.982  0.9814 0.9806 0.9802 0.9786 0.9778 0.9766 0.9758 0.9752
 0.9748 0.9738 0.972  0.9716 0.9708 0.9704]
mean test roc auo ovr: [0.94264863 0.98663311 0.99277442 0.99409249 0.99253886 0.99168005
 0.99108534 0.99094444 0.99033069 0.98983936 0.98875766 0.98835949
 0.98770723 0.98726485 0.9869287  0.98631701 0.98550929 0.98469667
 0.98402529 0.98333865 0.98301763 0.98242918 0.98207882 0.98150933
 0.98093509 0.98021469]
mean test f1 micro: [0.9876 0.988  0.9868 0.987  0.9856 0.985  0.984  0.9836 0.9834 0.9832
 0.9824 0.982  0.9814 0.9806 0.9802 0.9786 0.9778 0.9766 0.9758 0.9752
 0.9748 0.9738 0.972  0.9716 0.9708 0.9704]
    n_neighbors  score_acc
0             5     0.9880
1             1     0.9876
2            13     0.9870
3             9     0.9868
4            17     0.9856
5            21     0.9850
6            25     0.9840
7            29     0.9836
8            33     0.9834
9  

mean test accuracy: [0.99   0.9902 0.9888 0.9868 0.9846 0.9838 0.984  0.9842 0.9844 0.9838
 0.983  0.9836 0.983  0.982  0.9818 0.981  0.98   0.9806 0.9794 0.9786
 0.9766 0.9752 0.9736 0.9728 0.9722 0.9706]
mean test roc auo ovr: [0.92373851 0.98124193 0.99016238 0.99410507 0.99304448 0.99142774
 0.99014397 0.9888962  0.9878978  0.98669013 0.98498552 0.9841481
 0.98293572 0.98230412 0.98077635 0.97914336 0.97775382 0.9766052
 0.97540338 0.97423867 0.97295653 0.97154604 0.97058768 0.96943017
 0.96846207 0.96750977]
mean test f1 micro: [0.99   0.9902 0.9888 0.9868 0.9846 0.9838 0.984  0.9842 0.9844 0.9838
 0.983  0.9836 0.983  0.982  0.9818 0.981  0.98   0.9806 0.9794 0.9786
 0.9766 0.9752 0.9736 0.9728 0.9722 0.9706]
    n_neighbors  score_acc
0             5     0.9902
1             1     0.9900
2             9     0.9888
3            13     0.9868
4            17     0.9846
5            33     0.9844
6            29     0.9842
7            25     0.9840
8            37     0.9838
9    

mean test accuracy: [0.9892 0.9884 0.9878 0.9856 0.9828 0.982  0.9824 0.9828 0.9824 0.982
 0.9806 0.9804 0.9806 0.9804 0.9798 0.9798 0.9778 0.9774 0.9766 0.975
 0.9734 0.972  0.9712 0.969  0.9688 0.967 ]
mean test roc auo ovr: [0.92838417 0.98629717 0.99237907 0.9944223  0.99305987 0.99204667
 0.99126004 0.99073546 0.99001521 0.98902601 0.9880668  0.98737518
 0.98636943 0.98530961 0.98441272 0.98362105 0.98324241 0.98257153
 0.98174562 0.98082975 0.97994003 0.97913799 0.97855017 0.97795866
 0.977463   0.97675788]
mean test f1 micro: [0.9892 0.9884 0.9878 0.9856 0.9828 0.982  0.9824 0.9828 0.9824 0.982
 0.9806 0.9804 0.9806 0.9804 0.9798 0.9798 0.9778 0.9774 0.9766 0.975
 0.9734 0.972  0.9712 0.969  0.9688 0.967 ]
    n_neighbors  score_acc
0             1     0.9892
1             5     0.9884
2             9     0.9878
3            13     0.9856
4            17     0.9828
5            29     0.9828
6            25     0.9824
7            33     0.9824
8            21     0.9820
9      

mean test accuracy: [0.9902 0.9894 0.9886 0.988  0.9872 0.9858 0.984  0.9834 0.9824 0.9816
 0.9822 0.9816 0.9814 0.9816 0.9806 0.9802 0.9794 0.9788 0.9772 0.9758
 0.9756 0.9746 0.9738 0.9734 0.9724 0.9712]
mean test roc auo ovr: [0.92539189 0.98348605 0.9880302  0.98954587 0.98831547 0.98723778
 0.98599297 0.98489511 0.98394133 0.98295874 0.98407964 0.98348893
 0.98292992 0.98232769 0.98141136 0.98054691 0.97984382 0.97918107
 0.97797372 0.97712944 0.97627363 0.97544663 0.9746254  0.97415284
 0.97482135 0.9740491 ]
mean test f1 micro: [0.9902 0.9894 0.9886 0.988  0.9872 0.9858 0.984  0.9834 0.9824 0.9816
 0.9822 0.9816 0.9814 0.9816 0.9806 0.9802 0.9794 0.9788 0.9772 0.9758
 0.9756 0.9746 0.9738 0.9734 0.9724 0.9712]
    n_neighbors  score_acc
0             1     0.9902
1             5     0.9894
2             9     0.9886
3            13     0.9880
4            17     0.9872
5            21     0.9858
6            25     0.9840
7            29     0.9834
8            33     0.9824
9  

...

mean test accuracy: [0.9898 0.9882 0.9876 0.9864 0.9842 0.9834 0.9832 0.9828 0.9818 0.981
 0.9808 0.9802 0.9796 0.9784 0.9788 0.9782 0.9772 0.9764 0.9756 0.9748
 0.974  0.972  0.9714 0.9714 0.9696 0.9684]
mean test roc auo ovr: [0.93880686 0.98691227 0.99115639 0.9949407  0.9940556  0.99323316
 0.99255063 0.99145003 0.99065147 0.99032094 0.98921515 0.98845446
 0.98772226 0.98688725 0.98630086 0.98581038 0.98514895 0.98461613
 0.98398035 0.98308289 0.98257934 0.98184743 0.98101633 0.98040435
 0.97984048 0.97918488]
mean test f1 micro: [0.9898 0.9882 0.9876 0.9864 0.9842 0.9834 0.9832 0.9828 0.9818 0.981
 0.9808 0.9802 0.9796 0.9784 0.9788 0.9782 0.9772 0.9764 0.9756 0.9748
 0.974  0.972  0.9714 0.9714 0.9696 0.9684]
    n_neighbors  score_acc
0             1     0.9898
1             5     0.9882
2             9     0.9876
3            13     0.9864
4            17     0.9842
5            21     0.9834
6            25     0.9832
7            29     0.9828
8            33     0.9818
9    

mean test accuracy: [0.9876 0.987  0.9862 0.9848 0.984  0.9834 0.983  0.9828 0.9834 0.9828
 0.9828 0.983  0.9832 0.982  0.9832 0.982  0.9824 0.9826 0.9824 0.9816
 0.9804 0.9796 0.9788 0.9778 0.9774 0.9754]
mean test roc auo ovr: [0.93036437 0.98344184 0.98741657 0.98898129 0.99284933 0.99131743
 0.9902451  0.98923296 0.9877503  0.98671627 0.98532115 0.98452238
 0.98358135 0.98265127 0.98165007 0.98088412 0.98014553 0.97926469
 0.97830725 0.97795164 0.97721304 0.97630485 0.97551154 0.97500274
 0.97403436 0.9732137 ]
mean test f1 micro: [0.9876 0.987  0.9862 0.9848 0.984  0.9834 0.983  0.9828 0.9834 0.9828
 0.9828 0.983  0.9832 0.982  0.9832 0.982  0.9824 0.9826 0.9824 0.9816
 0.9804 0.9796 0.9788 0.9778 0.9774 0.9754]
    n_neighbors  score_acc
0             1     0.9876
1             5     0.9870
2             9     0.9862
3            13     0.9848
4            17     0.9840
5            33     0.9834
6            21     0.9834
7            49     0.9832
8            57     0.9832
9  

mean test accuracy: [0.989  0.9868 0.9854 0.9848 0.9852 0.9842 0.9838 0.9838 0.984  0.9834
 0.983  0.9822 0.982  0.9818 0.9808 0.9804 0.9782 0.9774 0.9766 0.9752
 0.9744 0.9738 0.9728 0.9726 0.9716 0.971 ]
mean test roc auo ovr: [0.92856439 0.9837154  0.98706368 0.99290404 0.99120254 0.99009738
 0.98861473 0.98732356 0.98631141 0.98496553 0.98363059 0.98230113
 0.98163366 0.98042455 0.97925375 0.97770544 0.97698873 0.97657293
 0.97562097 0.97520516 0.97368421 0.97289091 0.97226174 0.97146843
 0.97019368 0.96924718]
mean test f1 micro: [0.989  0.9868 0.9854 0.9848 0.9852 0.9842 0.9838 0.9838 0.984  0.9834
 0.983  0.9822 0.982  0.9818 0.9808 0.9804 0.9782 0.9774 0.9766 0.9752
 0.9744 0.9738 0.9728 0.9726 0.9716 0.971 ]
    n_neighbors  score_acc
0             1     0.9890
1             5     0.9868
2             9     0.9854
3            17     0.9852
4            13     0.9848
5            21     0.9842
6            33     0.9840
7            29     0.9838
8            25     0.9838
9  

mean test accuracy: [0.9866 0.9866 0.9872 0.9848 0.9848 0.985  0.985  0.9828 0.9826 0.982
 0.9824 0.9804 0.9808 0.9806 0.9786 0.9766 0.9772 0.9762 0.9748 0.9742
 0.9734 0.972  0.9708 0.9702 0.9682 0.9672]
mean test roc auo ovr: [0.92832752 0.98941542 0.99088232 0.99457637 0.99377212 0.99256515
 0.99212463 0.99157373 0.99106734 0.99014381 0.98965694 0.98917554
 0.98861499 0.98785219 0.98720491 0.98688893 0.98603299 0.98569128
 0.98504014 0.98429627 0.9835819  0.98320763 0.98261587 0.98203065
 0.98150489 0.98073851]
mean test f1 micro: [0.9866 0.9866 0.9872 0.9848 0.9848 0.985  0.985  0.9828 0.9826 0.982
 0.9824 0.9804 0.9808 0.9806 0.9786 0.9766 0.9772 0.9762 0.9748 0.9742
 0.9734 0.972  0.9708 0.9702 0.9682 0.9672]
    n_neighbors  score_acc
0             9     0.9872
1             1     0.9866
2             5     0.9866
3            21     0.9850
4            25     0.9850
5            13     0.9848
6            17     0.9848
7            29     0.9828
8            33     0.9826
9    

mean test accuracy: [0.9912 0.988  0.9872 0.9864 0.9856 0.984  0.983  0.9824 0.982  0.9826
 0.9824 0.9822 0.9824 0.9816 0.9814 0.9816 0.98   0.9792 0.9786 0.9786
 0.9774 0.9758 0.9748 0.9736 0.9728 0.9718]
mean test roc auo ovr: [0.95265099 0.98897533 0.99083967 0.98981097 0.98864108 0.98963232
 0.98868718 0.98786883 0.98688624 0.98632722 0.98540226 0.98435051
 0.98342554 0.98261007 0.984071   0.9836042  0.98273974 0.98204818
 0.9815929  0.98055556 0.97992162 0.97898801 0.97822729 0.97731097
 0.97651568 0.97567427]
mean test f1 micro: [0.9912 0.988  0.9872 0.9864 0.9856 0.984  0.983  0.9824 0.982  0.9826
 0.9824 0.9822 0.9824 0.9816 0.9814 0.9816 0.98   0.9792 0.9786 0.9786
 0.9774 0.9758 0.9748 0.9736 0.9728 0.9718]
    n_neighbors  score_acc
0             1     0.9912
1             5     0.9880
2             9     0.9872
3            13     0.9864
4            17     0.9856
5            21     0.9840
6            25     0.9830
7            37     0.9826
8            29     0.9824
9  

In [59]:
letter_knn_list

[array([0.9876, 0.987 , 0.9862, 0.9848, 0.984 , 0.9834, 0.983 , 0.9828,
        0.9834, 0.9828, 0.9828, 0.983 , 0.9832, 0.982 , 0.9832, 0.982 ,
        0.9824, 0.9826, 0.9824, 0.9816, 0.9804, 0.9796, 0.9788, 0.9778,
        0.9774, 0.9754]),
 array([0.9876, 0.987 , 0.9862, 0.9848, 0.984 , 0.9834, 0.983 , 0.9828,
        0.9834, 0.9828, 0.9828, 0.983 , 0.9832, 0.982 , 0.9832, 0.982 ,
        0.9824, 0.9826, 0.9824, 0.9816, 0.9804, 0.9796, 0.9788, 0.9778,
        0.9774, 0.9754]),
 array([0.9876, 0.987 , 0.9862, 0.9848, 0.984 , 0.9834, 0.983 , 0.9828,
        0.9834, 0.9828, 0.9828, 0.983 , 0.9832, 0.982 , 0.9832, 0.982 ,
        0.9824, 0.9826, 0.9824, 0.9816, 0.9804, 0.9796, 0.9788, 0.9778,
        0.9774, 0.9754]),
 array([0.9876, 0.987 , 0.9862, 0.9848, 0.984 , 0.9834, 0.983 , 0.9828,
        0.9834, 0.9828, 0.9828, 0.983 , 0.9832, 0.982 , 0.9832, 0.982 ,
        0.9824, 0.9826, 0.9824, 0.9816, 0.9804, 0.9796, 0.9788, 0.9778,
        0.9774, 0.9754]),
 array([0.9876, 0.987 , 0.9862, 

In [60]:
letter_knn_grid.cv_results_

{'mean_fit_time': array([0.00260286, 0.00220022, 0.00260005, 0.0021966 , 0.00239997,
        0.00260029, 0.0026    , 0.00259991, 0.00259681, 0.00199671,
        0.00239997, 0.00220022, 0.00299706, 0.00260444, 0.00240359,
        0.00299706, 0.00280004, 0.00220003, 0.0024003 , 0.00219998,
        0.00220003, 0.00300002, 0.00240006, 0.00279703, 0.00220022,
        0.00280032]),
 'std_fit_time': array([4.86342191e-04, 4.00137940e-04, 4.90349380e-04, 4.01522778e-04,
        4.90329760e-04, 4.90154472e-04, 4.90115662e-04, 4.90135100e-04,
        4.93759280e-04, 6.07925301e-06, 4.89843127e-04, 3.99780359e-04,
        6.30002768e-06, 4.93819371e-04, 4.87791049e-04, 6.32215003e-04,
        3.99971293e-04, 4.00114329e-04, 4.90544071e-04, 3.99422855e-04,
        4.00233734e-04, 1.50789149e-07, 4.90446746e-04, 3.98511548e-04,
        4.00018934e-04, 3.99994889e-04]),
 'mean_score_time': array([0.12779317, 0.18339381, 0.18519073, 0.19159398, 0.18679881,
        0.19179525, 0.1909936 , 0.19118991, 

In [61]:
letter_knn_grid.cv_results_['rank_test_accuracy']

array([ 1,  2,  3,  4,  5,  6,  7,  9, 13,  8,  9, 12,  9, 14, 16, 14, 17,
       18, 19, 19, 21, 22, 23, 24, 25, 26])

In [62]:
letter_knn_grid.cv_results_['params'][ np.argmin(letter_knn_grid.cv_results_['rank_test_accuracy'])]

{'n_neighbors': 1}

In [63]:
letter_knn_grid.cv_results_['params'][ np.argmin(letter_knn_grid.cv_results_['rank_test_roc_auc_ovr'])]

{'n_neighbors': 9}

In [64]:
letter_knn_results = pd.DataFrame(letter_knn_grid.cv_results_['params'])
letter_knn_results['score_acc'] = letter_knn_grid.cv_results_['mean_test_accuracy']
letter_knn_cols = letter_knn_results.columns.to_series().str.split('__').apply(lambda x: x[-1])
letter_knn_results.columns = letter_knn_cols
letter_knn_results = letter_knn_results.sort_values(by=['score_acc'], ascending=False, ignore_index=True)
letter_knn_results

,n_neighbors,score_acc
0,1,0.9912
1,5,0.9880
2,9,0.9872
3,13,0.9864
4,17,0.9856
5,21,0.9840
6,25,0.9830
7,37,0.9826
8,29,0.9824
9,41,0.9824


In [65]:
X_train, X_test, y_train, y_test = train_test_split(letter_data2, letter_data['lettr capital letter'], train_size=5000)
sc = StandardScaler()
X_train = pd.DataFrame(sc.fit_transform(X_train), columns=X_train.columns)
X_test = sc.transform(X_test)
letter_knn_best = KNeighborsClassifier(weights='distance', n_neighbors=9)
letter_knn_best.fit(X_train, y_train)
letter_pred_knn = letter_knn_best.predict(X_test)
print('KNN:', classification_report(y_test, letter_pred_knn))


KNN:               precision    recall  f1-score   support

           0       1.00      0.99      0.99     14460
           1       0.84      0.88      0.86       540

    accuracy                           0.99     15000
   macro avg       0.92      0.94      0.93     15000
weighted avg       0.99      0.99      0.99     15000

